In [103]:
pip install mesa matplotlib numpy pandas scikit-learn seaborn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [104]:
# Se importan las librerias necesarias
from mesa import Agent, Model 
from mesa.space import MultiGrid 
# SimultanousActivation permite que los agentes se activen al mismo tiempo
from mesa.time import SimultaneousActivation

from mesa.datacollection import DataCollector 

# Para crear una animación de cada paso del modelo 
%matplotlib inline 
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Librerías para el manejo de números
import numpy as np
import pandas as pd
import random

# Librerías para medición del tiempo
import time
import datetime

In [105]:
MAXVAL = 10000

#### Valores Iniciales Para Deddian ####


#Comida inicial
comidaDeddian = 150
#Metabolismo al moverse
metabolismoDeddian = 2
#Vida máxima
vidaDeddian = 50
#Unidades máximas de alimento (capacidad de estomago)
maxComidaDeddian = 200


#### Valores Iniciales Para Nonine ####


#Comida inicial
comidaNonine = 10
#Metabolismo al moverse
metabolismoNonine = 3
#Vida máxima
vidaNonine = 25
#Unidades máximas de alimento (capacidad de estomago)
maxComidaNonine = 45



#Edad mínima para cuchicuchi
edadCuchi = 10

#Probabilidad de cuchicuchi
probCuchi = 0.5

#Unidad minima de alimento para coito deddian (120), nonine (40)
ComidaParaCuchiNonine = 40
ComidaParaCuchiDeddian = 120



#Intervalo para la edad/ciclo
anomas = 1

In [106]:
def get_grid(model):
    grid = np.zeros( (model.grid.width, model.grid.height) )
    for (content, x, y) in model.grid.coord_iter():
        if (content == None):
            grid[x][y] = 0.60
        else:
            grid[x][y] = content.types
    return grid

In [107]:
#Agente para los nonine

# Capacidad máxima de comida: 45 unidades.
# Tasa de metabolismo: 3 unidades/ciclo.
# Probilidad de reproducción en un ambiente adecuada: 50%
# Edad mínima de reproducción: 10 ciclos.
# Requisito mínimo de alimento para reproducirse: 40 unidades.
# Edad máxima: 25 etapas.
# Comida inicial: 10
# Un nonine le da un valor de comida de 10 a un deddian cuando se lo come.


class Suciedad(Agent):

    NONINE = 0.5

    #Esta es la funcion para iniciar los valores

    def __init__(self, pos, unique_id, model, types):
        super().__init__(unique_id, model)
        self.x, self.y = pos
        self.vivo = True

        self.uid = unique_id
        self.types = types
        self.new_pos = None
    
    
       
    #Esta es la funcion de muerte definida en el model
    def muerte(self):
      self.model.muerte(self)
    
    
    def step(self):
      if not self.vivo:
        self.muerte()
        return

      

     
      
    

In [108]:
#Agente para los deddian


# Capacidad máxima de comida: 200 unidades.
# Tasa de metabolismo: 2 unidades/ciclo.
# Probilidad de reproducción en un ambiente adecuada: 50%
# Edad mínima de reproducción: 10 ciclos.
# Requisito mínimo de alimento para reproducirse: 120 unidades.
# Edad máxima: 50 etapas.
# Comida inicial: 150


class Limpiador(Agent):

    DEDDIAN = 1 #identificador

    def __init__(self, pos, unique_id, model, types):
        super().__init__(unique_id, model)
        self.x, self.y = pos

        self.vivo = True
        self.uid = unique_id
        self.types = types
        self.new_pos = None

    
              
    
    def step(self):
      if self.model.Alrededor[self.pos[0]][self.pos[1]] == 1:
        self.model.Alrededor[self.pos[0]][self.pos[1]] = 0
      else:
        neighborhood = self.model.grid.get_neighborhood(self.pos, moore = True, include_center = False)
        new_pos = self.random.choice(neighborhood)
        a = self.x + new_pos[0]
        b = self.y + new_pos[1]
        if (a >= 0 and a < self.model.grid.width and b >= 0 and b < self.model.grid.height):
          self.new_pos = new_pos
          self.model.grid.move_agent(self, self.new_pos)

        

    

In [109]:
def get_grid(model):
    """ Esta función nos permite obtener el estado de los diferentes agentes.
        *param* model : Modelo del que obtendrá la información. 
        *return* una matriz con la información del estado de cada uno de los agentes."""
    grid = np.zeros( (model.grid.width, model.grid.height) )
    for x in range (model.grid.width):
        for y in range (model.grid.height):
            if model.grid.is_cell_empty( (x, y) ) :
                grid[x][y] = model.Alrededor[x][y] * 2
            else:
                grid[x][y] = 1
    #print (grid)
    return grid

In [110]:
class Modelo(Model):
  def __init__(self, height, width, Nonine_acum, Deddian_acum):
        #grid para la simulacion de espacios

        self.grid = MultiGrid(width, height, True)
        #para simular la activación simultánea de todos los agentes.

        self.simul = SimultaneousActivation(self)
        self.Alrededor = np.zeros( (width, height) )
        self.Nonine_acum = Nonine_acum
        self.Deddian_acum = Deddian_acum
        self.uid = 0
        self.width = width
        self.height = height
        
        
        

       
        
        for i in range(Deddian_acum):
          # if((1,1) is empty_cells):
              deddian1 = Limpiador((1,1),self.nid(), self, 1)
              self.grid.place_agent(deddian1, (1,1))
              self.simul.add(deddian1)
      
        
        # for i in range(Nonine_acum):
        #    for x in range (model.grid.width):
        #       for y in range (model.grid.height):
        #         if model.grid.is_cell_empty((x,y)) :
        #           nonine1 = Suciedad((x,y),self.nid(), self, 0.5)
        #           # se le ingresa el type como identificador
        #           nonine1.types = nonine1.NONINE
        #           self.grid.place_agent(nonine1, (x,y))
        #           self.simul.add(nonine1)
                  
        for i in range(Nonine_acum):
           x = int(np.random.rand() * MAXVAL) % width
           y = int(np.random.rand() * MAXVAL) % height
           if self.Alrededor[x][y] == 0:
             self.Alrededor[x][y] = 1
                    

          # for i in range(Nonine_acum):
          #   emptyCell = random.choice(empty_cells)
          #   nonine1 = Nonine(emptyCell,self.nid(), self, 0, 0.5)
          #   # se le ingresa el type como identificador
          #   nonine1.types = nonine1.NONINE
          #   self.grid.place_agent(nonine1, emptyCell)
          #   self.simul.add(nonine1)
          #   empty_cells.remove(emptyCell)
        
        # Método para ingresar el pasto de nonines

        # cantidad = int((width * height))
        # for i in range(cantidad):
        #     finished = False
        #     while not finished:
        #         x = int(np.random.rand() * MAX) % width
        #         y = int(np.random.rand() * MAX) % height
        #         #define si el pasto ya ha sido ingrsado, por lo tanto finaliza 
        #         #while
        #         if self.Alrededor[x][y] == 0:
        #             self.Alrededor[x][y] = 20
        #             finished = True
        
        self.datacollector = DataCollector(model_reporters={"Grid": get_grid})


        #funcion para matar o destruir a cualquiera, urilizado en la clase de 
        #daddian y nonine
  
  def muerte(self,a ):
      self.Nonine_acum -= 1
      self.simul.remove(a)
      self.grid.remove_agent(a)
    
  #Con esta funcion los agentas ya pueden aparecer con un id respectivo
  def nid(self):
    uid = self.uid + 1
    self.uid  = uid
    return uid

 
  # def pasto(self):
  #   cantidad = int(self.width * self.height)
  #   for i in range(cantidad):
  #     finished = False
  #     while not finished:
  #       x = int(np.random.rand() * MAX) % self.width
  #       y = int(np.random.rand() * MAX) % self.height
  #       self.Alrededor[x][y] += 1
  #       finished = True
  
  def step(self):
      self.datacollector.collect(self)
      self.simul.step()

In [111]:


#Valores iniciales
#####################
HEIGHT = 35
WIDTH = 35
percentage = 25
limpiador= 3
######################


max_capacity = HEIGHT * WIDTH
total = int((percentage * max_capacity)/100)
suciedad = total

max_steps = 200
start_time = time.time()
i = 0
model = Modelo(HEIGHT, WIDTH, suciedad, limpiador)



while(i < max_steps):
  model.step()
  i += 1

print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

Tiempo de ejecución: 0:00:00.315551


In [112]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [113]:
%%capture

fig, axs = plt.subplots(figsize=(10,10))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap='YlOrRd')

def animate(i):
    patch.set_data(all_grid.iloc[i][0])

anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [114]:
anim